In [ ]:
import os
import sys
import time
import tempfile
import logging
import argparse
from lib import ApytramNeeds
from lib import BlastPlus
from lib import Trinity


start_time = time.time()

In [ ]:
### Option parsing
parser = argparse.ArgumentParser(prog = "apytram.py",
                                 description='''
    Run apytram on a fastq file to retrieve
    homologous sequences of bait sequences.''')

requiredOptions = parser.add_argument_group('required arguments')
requiredOptions.add_argument('-d', '--database', nargs='?', type=str,
                             help='Database preffix name', required=True)

parser.add_argument('--version', action='version', version='%(prog)s 1.0')
parser.add_argument('-log', nargs='?', type=str, default="apytram.log")
parser.add_argument('-t', '--tmp',  type=str,
                    help = "Directory to stock intermediary files for the apytram run. (default: a directory in /tmp which will be removed at the end)",
                    default = "" )


parser.add_argument('-fa', '--fasta',  type=str,
                   help = "Fasta formatted RNA-seq data to build the database of reads")
parser.add_argument('-fq', '--fastq',  type=str,
                   help = "Fastq formatted RNA-seq data to build the database of reads")
parser.add_argument('-out', '--output',  type=str, default = "./")



parser.add_argument('-q', '--query',  type=str,
                    help = "Fasta file (nt) with bait sequence for the apytram run." )
parser.add_argument('-i', '--iteration_max',  type=int,
                    help = "Maximum number of iteration. (Default 5)",
                    default = 5 )
parser.add_argument('-e', '--evalue',  type=float,
                    help = "Evalue. (Default 1e-3)",
                    default = 1e-3 )

parser.add_argument('--threads',  type=int,
                    help = "Available threads. (Default 1)",
                    default = 1 )


#args = parser.parse_args()
args = parser.parse_args('-d example_exec/db/examplefq -out Out_test -fq example/example_db.fastq -q example/ref_gene.fasta -t example_exec/tmp -i 5'.split())

MaxIteration = args.iteration_max
Threads = args.threads
Evalue = args.evalue


In [ ]:
### Set up the logger
LogFile = args.log
# create logger with 'spam_application'
logger = logging.getLogger('apytram')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler(LogFile)
fh.setLevel(logging.DEBUG)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)

In [ ]:
### Set up the working directory
if args.tmp:
    if os.path.isdir(args.tmp):
        logger.info("The temporary directory %s exists" %(args.tmp) )
    else:
        logger.info("The temporary directory %s does not exist, it will be created" % (args.tmp))
        os.makedirs(args.tmp)
    TmpDirName = args.tmp
else:
    TmpDirName = tempfile.mkdtemp()

# Remove only empty directory
#os.rmdir(TmpDirName)

In [ ]:
### Set up the output directory
if args.output:
    if os.path.isdir(args.output):
        logger.info("The output directory %s exists" %(args.output) )
    else:
        logger.info("The temporary directory %s does not exist, it will be created" % (args.output))
        os.makedirs(args.output)
    OutDirName = args.output
else:
    logger.error("The output directory must be defined")
    sys.exit(1)

In [ ]:
### Check that there is a database else built it
DatabaseName = args.database
if not os.path.isfile(DatabaseName+".nhr"):
    logger.info(DatabaseName+".nhr does not exist")
    #Build blast formated database from a fasta file
    if args.fastq or args.fasta:
        if args.fastq:
            if not os.path.isfile(args.fastq):
                logger.error("The fastq file (-fq) does not exist.")
                sys.exit(1)
            else:
                # Format the fastq file in fasta
                InputFasta = TmpDirName + "/" + os.path.basename(args.fastq) + ".fasta"
                ExitCode = ApytramNeeds.fastq2fasta(args.fastq,InputFasta)
        elif args.fasta:
            InputFasta = args.fasta
            
        if not os.path.isfile(InputFasta):
            logger.error("The fasta file (-fa) does not exist.")
            sys.exit(1)
        if os.path.isdir(os.path.dirname(DatabaseName)):
            logger.info("Database directory exists")
        else:
            logger.info("Database directory does not exist, we create it")
            os.makedirs(os.path.dirname(DatabaseName))
        # database building
        logger.info(DatabaseName + " database building")
        MakeblastdbProcess = BlastPlus.Makeblastdb(InputFasta,DatabaseName)
        ExitCode = MakeblastdbProcess.launch()
    else :
        logger.error("The database is not formatted ! A fasta file (-fa) or a fastq file (-fq) is required !")
        sys.exit(1)
if not os.path.isfile(DatabaseName+".nhr"):
    logger.error("Problem in the database building")
    logger.info(DatabaseName+".nhr does not exist")
    sys.exit(1)
else:
    logger.info(DatabaseName+".nhr exists")

In [ ]:
### If there is a query continue, else stop
if not args.query:
    logger.info("There is no query (-q), apytram have finished.")
    quit()
elif not os.path.isfile(args.query):
    logger.error(args.query+" (-q) is not a file.")
    sys.exit(1)
else:
    queryFile = args.query
    QueryDatabaseName = TmpDirName + "/" + queryFile.split(".")[0]
    BlastdbcmdProcess = BlastPlus.Makeblastdb(queryFile,QueryDatabaseName)
    command = BlastdbcmdProcess.launch()
    print command
    logger.info("apytram will run with \"%s\" as reads database and \"%s\" as bait sequences" %(DatabaseName,queryFile))
    

In [ ]:
### Make iterations
# Initialisation
i = 0
Stop = False
BaitSequences = queryFile

logger.info("Iterations begin")
while (i < MaxIteration) and (Stop == False):
    i+=1
    logger.info("Iteration %d/%d" %(i,MaxIteration))
    # Blast bait seqeunce on database of reads
    logger.info("Blast bait sequences on reads database")
    ReadNamesFile = TmpDirName + "/ReadNames.%d.txt" % (i)
    BlastnProcess = BlastPlus.Blast("blastn", DatabaseName, BaitSequences)
    BlastnProcess.Evalue = Evalue
    BlastnProcess.Threads = Threads
    BlastnProcess.OutFormat = "6 sacc"
    # Write read names in ReadNamesFile
    ExitCode = BlastnProcess.launch(ReadNamesFile)
    BlastnProcess.OutFormat = "6"
    ExitCode = BlastnProcess.launch(ReadNamesFile+".completetable")
    # Get paired reads names
    ExitCode = ApytramNeeds.add_paired_read_names(ReadNamesFile)
    # Retrieve sequences
    logger.info("Retrieve sequences")
    ReadFasta = TmpDirName + "/Reads.%d.fasta" % (i)
    BlastdbcmdProcess = BlastPlus.Blastdbcmd(DatabaseName, ReadNamesFile, ReadFasta)
    BlastdbcmdProcess.launch()
    # Launch Trinity
    logger.info("Launch Trinity")
    TrinityFasta = TmpDirName + "/Trinity_iter_%d" % (i)
    TrinityProcess = Trinity.Trinity(ReadFasta,TrinityFasta)
    # Use the  --full_cleanup Trinity option to keep only the contig file
    exitCode = TrinityProcess.launch("full_cleanup")
    if exitCode != 0: # Trinity found nothing
        logger.info("Trinity found nothing (ExitCode: %d)" %exitCode)
        Stop = True
    else:
        # Filter Trinity contigs to keep only homologous sequences of the reference genes
        logger.info("Compare Trinity results with query sequences")
        TrinityFasta = TrinityFasta + ".Trinity.fasta"
        TrinityBlast = TmpDirName + "/Trinity_iter_%d.blast" % (i)
        BlastnProcess = BlastPlus.Blast("blastn", QueryDatabaseName, TrinityFasta)
        BlastnProcess.OutFormat = "6"
        BlastnProcess.Evalue = 1e-8
        BlastnProcess.perc_identity = 80
        BlastnProcess.launch(TrinityBlast)
        # Filter hit
        FileteredTrinityFasta = TrinityFasta
        # Validated sequences become bait sequences
        BaitSequences = FileteredTrinityFasta
        # Compare to the previous iteration
    
    
# Write output
logger.info("End of Iterations")







In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))